# Signature Verification

In [10]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

In [11]:
def load_data(base_dir, csv_file):
    df = pd.read_csv(csv_file)
    images = []
    labels = []
    filenames = []

    for i, row in df.iterrows():
        image_path = os.path.join(base_dir, row['file_name'] + '.jpg')
        image = cv2.imread(image_path)
        if image is None:
            print(f"Warning: {image_path} not found.")
            continue  # Skip this iteration if the image is not found
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image) / 255.0  # Normalize image data
        images.append(image)
        labels.append(1 if row['signature'] == 'Y' else 0)
        filenames.append(row['file_name'] + '.jpg')

    images = np.array(images)
    labels = np.array(labels)
    return images, labels, filenames

In [13]:
base_dir = '../example_data/output4/timestamps/'
csv_file = '../example_data/ground_truths/train_sign_output4.csv'

images, labels, filenames = load_data(base_dir, csv_file)

In [14]:
X_train, X_test, y_train, y_test, filenames_train, filenames_test = train_test_split(images, labels, filenames, test_size=0.20, random_state=42)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
checkpoint = ModelCheckpoint(
    '../output_models/best_signature_vs_text_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

In [17]:
model.fit(
    X_train, y_train, 
    epochs=20, 
    validation_data=(X_test, y_test), 
    batch_size=32, 
    callbacks=[checkpoint]
)

Epoch 1/20
23/23 [==============================] - ETA: 0s - loss: 0.4625 - accuracy: 0.7654
Epoch 1: val_accuracy improved from -inf to 0.82222, saving model to ../output_models\best_signature_vs_text_model.h5
23/23 [==============================] - 3s 109ms/step - loss: 0.4625 - accuracy: 0.7654 - val_loss: 0.3760 - val_accuracy: 0.8222
Epoch 2/20
 1/23 [>.............................] - ETA: 2s - loss: 0.2636 - accuracy: 0.9062

d:\Development\Python-Mini-Projects\data_extraction\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


23/23 [==============================] - ETA: 0s - loss: 0.3196 - accuracy: 0.8645
Epoch 2: val_accuracy improved from 0.82222 to 0.83889, saving model to ../output_models\best_signature_vs_text_model.h5
23/23 [==============================] - 3s 110ms/step - loss: 0.3196 - accuracy: 0.8645 - val_loss: 0.3319 - val_accuracy: 0.8389
Epoch 3/20
23/23 [==============================] - ETA: 0s - loss: 0.2962 - accuracy: 0.8673
Epoch 3: val_accuracy improved from 0.83889 to 0.85556, saving model to ../output_models\best_signature_vs_text_model.h5
23/23 [==============================] - 2s 105ms/step - loss: 0.2962 - accuracy: 0.8673 - val_loss: 0.3510 - val_accuracy: 0.8556
Epoch 4/20
23/23 [==============================] - ETA: 0s - loss: 0.2630 - accuracy: 0.8841
Epoch 4: val_accuracy improved from 0.85556 to 0.86667, saving model to ../output_models\best_signature_vs_text_model.h5
23/23 [==============================] - 2s 106ms/step - loss: 0.2630 - accuracy: 0.8841 - val_loss: 0.3

In [18]:
model.load_weights('../output_models/best_signature_vs_text_model.h5')

In [19]:

predictions = (model.predict(X_test) > 0.95).astype('int32')

# Save results to CSV
results_df = pd.DataFrame({
    'Filename': filenames_test,
    'Ground Truth': y_test,
    'Predicted': predictions.flatten()
})
results_df.to_csv('../output_csv/test_predictions.csv', index=False)

print(classification_report(y_test, predictions))
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))

6/6 [==============================] - 0s 19ms/step
              precision    recall  f1-score   support

           0       0.65      0.98      0.78        50
           1       0.99      0.80      0.89       130

    accuracy                           0.85       180
   macro avg       0.82      0.89      0.83       180
weighted avg       0.90      0.85      0.86       180

Confusion Matrix:
[[ 49   1]
 [ 26 104]]
